In [1]:
import numpy as np
import matplotlib.pyplot as plt

from unet_model import UNet
from dataset import *
from quantitative_results import *

In [2]:
config = {
  'seed': 310, # random seed
  'input_shape': (240, 360), # size of input images (height, width)
  'output_shape': (240, 360), # size of output target
  'n_train': 40, # number of training images per view
  'batch_size': 2, # number of images per optimization step
  'lr': 1e-2, # learning rate
  'n_epochs': 50, # number of passes through training data
  # optionally perform random cropping, specify integer < max(H, W) to use cropping for training
  'crop_size': 'None',
  # fraction of training epochs to lower learning rate by 1/10, e.g. [0.6, 0.8]
  # lowers learning rate at epochs 120 and 160 if we have 200 training epochs
  'milestones': [0.8],
  'views': ['Almond at Washington North'], # list of views for training  'Almond at Washington East'
  'images_path': 'Annotations-Images', # path to RGB images
  'gt_path': 'Annotations-GT', # path to ground-truth segmentation masks
  'log_path': 'x' # path to directory you make for saving results
}

# set up model
n_channels = 3 # RGB images
model = UNet(n_channels)

# set up datasets
seed = config['seed']
np.random.seed(seed) 
views = config['views']
logging_path = config['log_path']
if not os.path.exists(logging_path):
    os.mkdir(logging_path)

In [3]:
# make initial datasets for each view
print('Loading datasets...')
datasets = []
for v in views:
    datasets.append(BFSEvaluationDataset(config['images_path'],
                                         config['gt_path'],
                                         [v],
                                         config['input_shape'],
                                         config['output_shape']))

Loading datasets...


In [4]:
from active_learning import *
act = ActiveLearning(model=model, n_init=20 * len(config['views']), n_train_per_view=40, datasets=datasets, config=config)

100%|██████████| 50/50 [00:54<00:00,  1.09s/it]


In [5]:
accuracies = act.run_loop(5)

running loop
Metrics with 20 training images
Precision = 0.936 | Recall = 0.735 | F-measure = 0.823 | Blob recall = 0.648

new acquisition version
Uncertains:  [  75  452  212  784  613  628  833   96  865   19 1279  184  113  717
   58  103  426  400  241  261]
Chosen uncertainties:  [13  3  6  8 10]


100%|██████████| 50/50 [01:06<00:00,  1.33s/it]

Metrics with 20 training images
Precision = 0.944 | Recall = 0.811 | F-measure = 0.872 | Blob recall = 0.796

